## Segmenting and clustering neighboorhoods in Toronto

In [1]:
# importing dependencies
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import io
import geocoder
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

## Start the DataFrame

Using only pandas and the pd.read_html() method. 

In [2]:
# Another method
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
page = bs(req.content)
table = page.find('table')
df = pd.read_html(str(table), header = 0)[0]
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [3]:
# Ignoring the "Not assigned" rows

df = df[df.Borough != 'Not assigned']
df = pd.DataFrame(df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join))
df = df.reset_index()
df.Neighborhood[df.Neighborhood == 'Not assigned'] = df.Borough
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
# Checking the M5A Postal code
print('Verify that the \'M5A\'-case is correct : \n{}\n '.format(df[df['Postal Code'] == 'M5A']))


Verify that the 'M5A'-case is correct : 
   Postal Code           Borough               Neighborhood
53         M5A  Downtown Toronto  Regent Park, Harbourfront
 


## Last requirement: 

Use the .shape method to print the number of rows of your dataframe.use the .shape method to print the number of rows of your dataframe.

In [5]:
print('The number of rows of the dataframe is {}'.format(df.shape[0]))

The number of rows of the dataframe is 103


## Use the csv

Geocoder was taking too long to get the coordinates, so I used the url made available on Coursera. 

In [6]:
url= "http://cocl.us/Geospatial_data"
req = requests.get(url).content
coord = pd.read_csv(io.StringIO(req.decode('utf-8')))
coord.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


## Merging the coordinates with the DataFrame with the Boroughs and Neighborhoods of Toronto. 

In [7]:
df_merged = pd.merge(left = df, right = coord, left_on='Postal Code', right_on='Postal Code', how = 'left')
df_merged.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
# Convert the coordinates to numeric
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Map of Toronto with neighborhoods superimposed



In [9]:


neighborhoods = df_merged
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# # add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto


## Using the Foursquare API to veg the venues and segment the neighborhoods



In [13]:
# Define my credentials. These objects will be deleted for the upload on github

CLIENT_ID, CLIENT_SECRET, VERSION = pd.read_csv('credentials.csv')


In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


## Creating a function to make the URL of the request



In [15]:
def get4sqURL(lat, lng, LIMIT = 100, radius=500):
    '''
    This function takes latitude, longitude, maximum number of searches and radius and returns the url for that search on the FourSquare API
    '''

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
    return url

In [16]:
def get_category_type(row):
    '''
    This function takes as input the row of the json file returned by the Foursquare API and outputs the category and name of the venue it describes. 
    '''
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = get4sqURL(lat,lng,radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood']+' '+neighborhoods['Postal Code']


neighborhoods.head()
neighborhoods['Neighborhood'].size

103

In [19]:
# DataFrame containing all the venues listed in Toronto area

toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'], 
                                   latitudes=neighborhoods['Latitude'], 
                                   longitudes=neighborhoods['Longitude'], 
                                   radius=4000)

Malvern, Rouge M1B
Rouge Hill, Port Union, Highland Creek M1C
Guildwood, Morningside, West Hill M1E
Woburn M1G
Cedarbrae M1H
Scarborough Village M1J
Kennedy Park, Ionview, East Birchmount Park M1K
Golden Mile, Clairlea, Oakridge M1L
Cliffside, Cliffcrest, Scarborough Village West M1M
Birch Cliff, Cliffside West M1N
Dorset Park, Wexford Heights, Scarborough Town Centre M1P
Wexford, Maryvale M1R
Agincourt M1S
Clarks Corners, Tam O'Shanter, Sullivan M1T
Milliken, Agincourt North, Steeles East, L'Amoreaux East M1V
Steeles West, L'Amoreaux West M1W
Upper Rouge M1X
Hillcrest Village M2H
Fairview, Henry Farm, Oriole M2J
Bayview Village M2K
York Mills, Silver Hills M2L
Willowdale, Newtonbrook M2M
Willowdale, Willowdale East M2N
York Mills West M2P
Willowdale, Willowdale West M2R
Parkwoods M3A
Don Mills M3B
Don Mills M3C
Bathurst Manor, Wilson Heights, Downsview North M3H
Northwood Park, York University M3J
Downsview M3K
Downsview M3L
Downsview M3M
Downsview M3N
Victoria Village M4A
Parkview Hi

In [20]:
toronto_venues.shape

(2130, 7)

In [21]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge M1B",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek M1C",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill M1E",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill M1E",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill M1E",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
5,"Guildwood, Morningside, West Hill M1E",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
6,"Guildwood, Morningside, West Hill M1E",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,"Guildwood, Morningside, West Hill M1E",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
8,"Guildwood, Morningside, West Hill M1E",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,Woburn M1G,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop


In [22]:
print('There are {} unique categories of venues in the {} neighborhoods of Toronto.'.format(len(toronto_venues['Venue Category'].unique()), len(toronto_venues['Neighborhood'].unique())))

There are 272 unique categories of venues in the 99 neighborhoods of Toronto.


### Create a one-hot-encoded dataframe of all venues in Toronto

In [52]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
toronto_onehot.shape

(2130, 272)

In [64]:
# group de rows by neighborhood and take the mean of the frequency for the occurence of each category.abs

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt M1S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch M8W",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview Nort...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village M2K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East M5M",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

                  venue  freq
0          Clothing Store  0.20
1       Accessories Store  0.13
2  Furniture / Home Store  0.13
3           Women's Store  0.07
4             Event Space  0.07
5      Miscellaneous Shop  0.07
6      Athletics & Sports  0.07
7     Arts & Crafts Store  0.07
8                Boutique  0.07
9   Vietnamese Restaurant  0.07


----Lawrence Park M4N----
             venue  freq
0           Lawyer  0.25
1      Swim School  0.25
2         Bus Line  0.25
3             Park  0.25
4     Optical Shop  0.00
5  Organic Grocery  0.00
6     Liquor Store  0.00
7           Lounge  0.00
8    Luggage Store  0.00
9           Market  0.00


----Leaside M4G----
                    venue  freq
0             Coffee Shop  0.09
1            Burger Joint  0.06
2  Furniture / Home Store  0.06
3     Sporting Goods Shop  0.06
4                    Bank  0.06
5                 Brewery  0.03
6              Sports Bar  0.03
7              Beer Store  0.03
8               Bike Shop  0.03
9    

### Define a function to sort the venues listed in descending order

In [125]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [126]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt M1S,Latin American Restaurant,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Colombian Restaurant,Comfort Food Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
1,"Alderwood, Long Branch M8W",Pizza Place,Skating Rink,Coffee Shop,Gym,Pub,Pool,Sandwich Place,Dim Sum Restaurant,Dance Studio,Deli / Bodega
2,"Bathurst Manor, Wilson Heights, Downsview Nort...",Coffee Shop,Bank,Sushi Restaurant,Mobile Phone Shop,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Restaurant,Bridal Shop
3,Bayview Village M2K,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
4,"Bedford Park, Lawrence Manor East M5M",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Grocery Store,Pharmacy,Juice Bar,Liquor Store,Indian Restaurant,Pub


# Clustering the Neighborhoods

In [127]:

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [128]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#toronto_grouped_clustering.head()


array([0, 4, 4, 0, 4, 4, 0, 4, 0, 0], dtype=int32)

### Add the cluster number to the DataFrame

In [133]:

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


toronto_merged = neighborhoods


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge M1B",43.806686,-79.194353,0.0,Fast Food Restaurant,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek M1C",43.784535,-79.160497,3.0,Bar,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Ethiopian Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill M1E",43.763573,-79.188711,0.0,Electronics Store,Rental Car Location,Breakfast Spot,Intersection,Bank,Medical Center,Mexican Restaurant,Concert Hall,Dessert Shop,Drugstore
3,M1G,Scarborough,Woburn M1G,43.770992,-79.216917,4.0,Coffee Shop,Korean Restaurant,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
4,M1H,Scarborough,Cedarbrae M1H,43.773136,-79.239476,0.0,Fried Chicken Joint,Caribbean Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Gas Station,Hakka Restaurant,Dog Run,Distribution Center


In [134]:
# toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
# toronto_merged.head()
toronto_merged = toronto_merged.dropna()

In [135]:
 # create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Cluster #0

In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,Scarborough,0.0,Electronics Store,Rental Car Location,Breakfast Spot,Intersection,Bank,Medical Center,Mexican Restaurant,Concert Hall,Dessert Shop,Drugstore
4,Scarborough,0.0,Fried Chicken Joint,Caribbean Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Gas Station,Hakka Restaurant,Dog Run,Distribution Center
5,Scarborough,0.0,Grocery Store,Playground,Convenience Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
7,Scarborough,0.0,Bakery,Metro Station,Soccer Field,Ice Cream Shop,Intersection,Bus Station,Bus Line,Park,Dog Run,Distribution Center
